In [1]:
import yfinance as yf
import numpy as np


# Download historical data for required stocks
tickers = ["MSFT","AAPL","GOOG"]
ohlcv_data = {}

# looping over tickers and storing OHLCV dataframe in dictionary
for ticker in tickers:
    temp = yf.download(ticker,period='1mo',interval='15m')
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] = temp

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [15]:
def MACD(DF,a=12 ,b=26 ,c=9):
    df = DF.copy()
    df["ma_fast"] = df["Adj Close"].ewm(span=a, min_periods=a).mean()
    df["ma_slow"] = df["Adj Close"].ewm(span=b, min_periods=b).mean()
    df["macd"] = df["ma_fast"] - df["ma_slow"]
    df["signal"] = df["macd"].ewm(span = c , min_periods=c).mean()
    return df.loc[:,["macd","signal"]]

for ticker in ohlcv_data:
    ohlcv_data[ticker][["MACD","SIGNAL"]] = MACD(ohlcv_data[ticker])

print(ohlcv_data["MSFT"].tail())

                           Open        High         Low       Close  \
Datetime                                                              
2023-12-12 14:45:00  372.579987  372.720001  372.370514  372.572113   
2023-12-12 15:00:00  372.589996  373.059998  372.531403  372.839996   
2023-12-12 15:15:00  372.850006  373.109985  372.804504  372.959991   
2023-12-12 15:30:00  372.940002  373.116913  372.625000  373.100006   
2023-12-12 15:45:00  373.100006  374.420013  372.920013  374.339996   

                      Adj Close   Volume   ATR      MACD    SIGNAL  
Datetime                                                            
2023-12-12 14:45:00  372.572113   453874  None  0.477178  0.489595  
2023-12-12 15:00:00  372.839996   844978  None  0.484557  0.488587  
2023-12-12 15:15:00  372.959991   466131  None  0.494389  0.489748  
2023-12-12 15:30:00  373.100006   556978  None  0.507627  0.493323  
2023-12-12 15:45:00  374.339996  2515852  None  0.611130  0.516885  


In [19]:
# ATR INDICATOR 

def ATR(DF,n=14):
    df=DF.copy()
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = df["High"] - df["Adj Close"].shift(1)
    df["L-PC"] = df["Low"] - df["Adj Close"].shift(1)
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1,skipna=False)
    df["ATR"] = df["TR"].ewm(span = n , min_periods = n).mean()
    return df["ATR"]

for ticker in ohlcv_data:
    ohlcv_data[ticker]["ATR"] = ATR(ohlcv_data[ticker])

print(ohlcv_data["MSFT"].tail(5))


                           Open        High         Low       Close  \
Datetime                                                              
2023-12-12 14:45:00  372.579987  372.720001  372.370514  372.572113   
2023-12-12 15:00:00  372.589996  373.059998  372.531403  372.839996   
2023-12-12 15:15:00  372.850006  373.109985  372.804504  372.959991   
2023-12-12 15:30:00  372.940002  373.116913  372.625000  373.100006   
2023-12-12 15:45:00  373.100006  374.420013  372.920013  374.339996   

                      Adj Close   Volume       ATR      MACD    SIGNAL  
Datetime                                                                
2023-12-12 14:45:00  372.572113   453874  0.558279  0.477178  0.489595  
2023-12-12 15:00:00  372.839996   844978  0.554321  0.484557  0.488587  
2023-12-12 15:15:00  372.959991   466131  0.521143  0.494389  0.489748  
2023-12-12 15:30:00  373.100006   556978  0.517245  0.507627  0.493323  
2023-12-12 15:45:00  374.339996  2515852  0.648279  0.611130  0.

In [23]:
#Bollinger band function

def Boll_band(DF,n=14):
    df = DF.copy()
    df["MB"] = df["Adj Close"].rolling(n).mean()
    df["UB"] = df["MB"] + 2*df["Adj Close"].rolling(n).std(ddof=0)
    df["LB"] = df["MB"] - 2*df["Adj Close"].rolling(n).std(ddof=0)
    df["BB_width"] = df["UB"] - df["LB"]
    return df[["MB","UB","LB","BB_width"]]

for ticker in ohlcv_data: 
    ohlcv_data[ticker][["MB","UB","LB","BB_width"]] = Boll_band(ohlcv_data[ticker])

print(ohlcv_data["MSFT"].tail(5))

                           Open        High         Low       Close  \
Datetime                                                              
2023-12-12 14:45:00  372.579987  372.720001  372.370514  372.572113   
2023-12-12 15:00:00  372.589996  373.059998  372.531403  372.839996   
2023-12-12 15:15:00  372.850006  373.109985  372.804504  372.959991   
2023-12-12 15:30:00  372.940002  373.116913  372.625000  373.100006   
2023-12-12 15:45:00  373.100006  374.420013  372.920013  374.339996   

                      Adj Close   Volume          MB          UB          LB  \
Datetime                                                                       
2023-12-12 14:45:00  372.572113   453874  372.290874  372.843854  371.737894   
2023-12-12 15:00:00  372.839996   844978  372.318730  372.936468  371.700992   
2023-12-12 15:15:00  372.959991   466131  372.388016  373.058099  371.717933   
2023-12-12 15:30:00  373.100006   556978  372.495159  373.103856  371.886461   
2023-12-12 15:45:00  3

In [4]:
#RSI indicator 

def RSI (DF,n=14):
    df=DF.copy()
    df["change"] = df["Adj Close"] - df["Adj Close"].shift(1)
    df["gain"] = np.where(df["change"]>=0,df["change"],0)
    df["loss"] = np.where(df["change"]<0,-1*df["change"],0)
    df["avgGain"] = df["gain"].ewm(alpha = 1/n , min_periods=n).mean()
    df["avgLoss"] = df["loss"].ewm(alpha = 1/n , min_periods=n).mean()
    df["rs"] = df["avgGain"] / df["avgLoss"] 
    df["rsi"] = 100 - (100/(1+df["rs"]))
    return df["rsi"]

for ticker in ohlcv_data:
    ohlcv_data[ticker]["RSI"] = RSI(ohlcv_data[ticker])

print(ohlcv_data["MSFT"].tail(5))

                           Open        High         Low       Close  \
Datetime                                                              
2023-12-14 14:45:00  365.580811  366.179993  365.459991  365.565002   
2023-12-14 15:00:00  365.570007  365.850006  364.994995  365.266693   
2023-12-14 15:15:00  365.269989  365.519989  364.890015  365.153595   
2023-12-14 15:30:00  365.160004  365.940002  365.000000  365.799988   
2023-12-14 15:45:00  365.790009  366.269989  364.929993  365.950012   

                      Adj Close   Volume        RSI  
Datetime                                             
2023-12-14 14:45:00  365.565002   847787  33.407627  
2023-12-14 15:00:00  365.266693  1052431  32.006920  
2023-12-14 15:15:00  365.153595  1210479  31.468220  
2023-12-14 15:30:00  365.799988  1157013  37.901201  
2023-12-14 15:45:00  365.950012  4333635  39.324780  


In [5]:
#ADX implementation python 


def ATR(DF,n=14):
    df=DF.copy()
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = df["High"] - df["Adj Close"].shift(1)
    df["L-PC"] = df["Low"] - df["Adj Close"].shift(1)
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1,skipna=False)
    df["ATR"] = df["TR"].ewm(span = n , min_periods = n).mean()
    return df["ATR"]

def ADX(DF, n=20):
    df= DF.copy()
    df["ATR"] = ATR(df,n)
    df["upmove"] = df["High"] - df["High"].shift(1)
    df["downmove"] = df["Low"].shift(1) - df["Low"]
    df["+dm"] = np.where((df["upmove"]>df["downmove"]) & (df["upmove"]>0 ),df["upmove"],0)
    df["-dm"] = np.where((df["downmove"]>df["upmove"]) & (df["downmove"]>0 ),df["downmove"],0)
    df["+di"] = 100 * (df["+dm"]/df["ATR"]).ewm(span= n , min_periods=n).mean()
    df["-di"] = 100 * (df["-dm"]/df["ATR"]).ewm(span= n , min_periods=n).mean()
    df["ADX"] = 100 * abs((df["+di"] -  df["-di"])/(df["+di"] + df["-di"])).ewm(span= n , min_periods=n).mean()
    return df["ADX"]

for ticker in ohlcv_data:
    ohlcv_data[ticker]["ADX"] = ADX(ohlcv_data[ticker])

print(ohlcv_data["MSFT"].tail(5))



                           Open        High         Low       Close  \
Datetime                                                              
2023-12-18 09:45:00  369.709991  369.950012  368.709991  369.760010   
2023-12-18 10:00:00  369.790009  371.119995  369.630005  370.619995   
2023-12-18 10:15:00  370.619995  372.709991  370.609985  372.489990   
2023-12-18 10:30:00  372.459991  372.660004  370.910004  370.910004   
2023-12-18 10:45:00  370.910004  370.975006  370.619995  370.720001   

                      Adj Close   Volume        ADX  
Datetime                                             
2023-12-18 09:45:00  369.760010   888237  26.457244  
2023-12-18 10:00:00  370.619995  1161501  25.278476  
2023-12-18 10:15:00  372.489990   901853  23.609545  
2023-12-18 10:30:00  370.910004   655469  22.099561  
2023-12-18 10:45:00  370.720001   214489  20.289564  


In [ ]:
# Renko chart 

